# Performing Required Imports

In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.datasets import mnist
import numpy as np
from keras.optimizers import SGD
from keras.utils import np_utils
import cv2

Using TensorFlow backend.


# Loading MNIST Dataset

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

# Adding an extra dimension to make the input inline with the model

In [3]:
x_train = x_train[:,:,:,np.newaxis]
x_test = x_test[:,:,:,np.newaxis]

# Transform the training and test labels into vectors in the range [0, classes] -- This generates a vector for each label, where the index of the label is set to 1 and all other entries to 0; in the case of MNIST, there are 10 class labels

In [4]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Checking dimensions of data

In [5]:
np.shape(x_train), np.shape(y_train), np.shape(x_test), np.shape(y_test)

((60000, 28, 28, 1), (60000, 10), (10000, 28, 28, 1), (10000, 10))

# Architecture

In [6]:
model = Sequential()

# first set of CONV => RELU => POOL
model.add(Convolution2D(6, 3, 3, border_mode="same", input_shape=(28, 28, 1)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL
model.add(Convolution2D(16, 5, 5, border_mode="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# first set of CONV => DROPOUT => RELU => POOL
model.add(Convolution2D(46, 7, 7, border_mode="same"))
model.add(Dropout(0.25))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# first set of FC => softmax
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=True), metrics=["accuracy"])

/home/talha/anaconda/envs/conda_env/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (3, 3), padding="same", input_shape=(28, 28, 1...)`
  after removing the cwd from sys.path.
/home/talha/anaconda/envs/conda_env/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), padding="same")`
  if __name__ == '__main__':
/home/talha/anaconda/envs/conda_env/lib/python2.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(46, (7, 7), padding="same")`
  


# Training The Model

In [7]:
model.fit(x_train, y_train, batch_size=128, nb_epoch=30, verbose=1)

/home/talha/anaconda/envs/conda_env/lib/python2.7/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
60000/60000 [==============================] - 5s 90us/step - loss: 1.3812 - acc: 0.7396
Epoch 2/30
60000/60000 [==============================] - 4s 67us/step - loss: 0.2772 - acc: 0.9126
Epoch 3/30
60000/60000 [==============================] - 4s 68us/step - loss: 0.2028 - acc: 0.9355
Epoch 4/30
60000/60000 [==============================] - 4s 68us/step - loss: 0.1648 - acc: 0.9483
Epoch 5/30
60000/60000 [==============================] - 4s 68us/step - loss: 0.1424 - acc: 0.9549
Epoch 6/30
60000/60000 [==============================] - 4s 68us/step - loss: 0.1270 - acc: 0.9598
Epoch 7/30
60000/60000 [==============================] - 4s 68us/step - loss: 0.1165 - acc: 0.9627
Epoch 8/30
60000/60000 [==============================] - 4s 67us/step - loss: 0.1056 - acc: 0.9658
Epoch 9/30
60000/60000 [==============================] - 4s 67us/step - loss: 0.0974 - acc: 0.9690
Epoch 10/30
60000/60000 [==============================] - 4s 68us/step - loss: 0.0904 - acc: 0.9713

# Testing The Model

In [8]:
(loss, accuracy) = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 0s 30us/step
accuracy: 98.37%


# Visualizing output in the form of image

In [26]:
# randomly select a few testing digits
for i in np.random.choice(np.arange(0, len(y_test)), size=(10,)):
    # classify the digit
    probs = model.predict(x_test[np.newaxis, i])
    prediction = probs.argmax(axis=1)

    # resize the image to 96 x 96
    image = (x_test[i][0] * 255).astype("uint8")
    image = cv2.merge([image] * 3)
    image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
    cv2.putText(image, str(prediction[0]), (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # show the image and prediction
    print(" Predicted: {}, Actual: {}".format(prediction[0], np.argmax(y_test[i])))
    cv2.imshow("Digit", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()    

 Predicted: 6, Actual: 6
 Predicted: 5, Actual: 5
 Predicted: 1, Actual: 1
 Predicted: 3, Actual: 3
 Predicted: 7, Actual: 7
 Predicted: 0, Actual: 0
 Predicted: 1, Actual: 1
 Predicted: 4, Actual: 4
 Predicted: 7, Actual: 7
 Predicted: 3, Actual: 3


# Conclusion

### The Model has 3 Convolution Layers using 6, 16 and 46 filters of sizes 3x3, 5x5 and 7x7 respectively each with Relu and Max pooling with  Dropout of 0.25 after the last convolution layer. Learning Rate is 0.0001 with nestrov momentum 0.9 using SGD as optimizer and utilizing categorical_crossentropy as loss while accuracy as a metric. The model has a Batch  Size of 128 with 30 epochs providing a Training Accuracy of 98.66% while Test Accuracy of 98.37%